contact me in telegram [alex seikin](https://t.me/seikin_alexey) 

## Задание
У вас SQL база с таблицами магазина, который существует много лет:
1) Users(userId, age)
2) Purchases (purchaseId, userId, itemId, date)
3) Items (itemId, price).

Напишите SQL запросы для расчета следующих метрик:

А) какую сумму в среднем в месяц тратит:
  - пользователи в возрастном диапазоне от 18 до 25 лет включительно
  - пользователи в возрастном диапазоне от 26 до 35 лет включительно

Б) в каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая

В) Какой товар дает наибольший вклад в выручку за последний год

Г) топ-3 товаров по выручке и их доля в общей выручке за любой год

Будет здорово, если ваше решение оформлено в виде работающего кода и основательно протестировано на придуманных данных (код наполнения бд НЕ прикладывайте). 
Для тестирования можно использовать онлайн-редактор https://sqliteonline.com/
Предпочтительный диалект - PostgreSQL, но можете использовать любой из доступных.

## План работы

1. создать бд и подключится к ней
2. импортировать таблицы с игрушечными данными в бд
3. выполнить SQL запросы из задания

## Импорты

In [52]:
import pandas as pd
from sqlalchemy import create_engine
import csv
from io import StringIO

## Вспомогательные функции

In [53]:
import csv
from io import StringIO

from sqlalchemy import create_engine

def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

In [54]:
# создание подключения к базе данных PostgresSQL
db_host = 'surus.db.elephantsql.com'
db_name = 'xftllppj'
db_user_name = 'xftllppj'
db_pass = 'nSZxkk4NKFagwgxRE8YsRjKO0j3EWUbW'

connecton_ps = create_engine(f'postgresql+psycopg2://{db_user_name}:{db_pass}@{db_host}/{db_name}')
#connecton_ps

In [55]:
# запрос к бд
def select(sql):
  return pd.read_sql(sql, connecton_ps)

 ## Импорт датасетов в переменные

In [56]:
# Импорт датасета с пользователями
users = pd.read_csv('datasets/who_is_blogger_dataset_users.csv')
users.head(5)

,user_id,age
0,570,33.0
1,580,43.0
2,568,52.0
3,829,35.0
4,413,28.0


In [57]:
# Импорт датасета с товарами
items = pd.read_csv('datasets/who_is_blogger_dataset_items.csv')
items.head(5)

,item_id,price
0,1,161.38
1,2,35.34
2,3,146.50
3,4,119.21
4,5,105.24


In [58]:
# Импорт датасета с покупками с преобразованием даты к типу datetime64[ns]
purchases = pd.read_csv('datasets/who_is_blogger_dataset_purchases.csv')
purchases['date'] = pd.to_datetime(purchases['date'], format='%Y-%m-%d %H:%M:%S')
display(purchases.dtypes)
display(purchases.head(5))

purchase_id             int64
item_id                 int64
user_id                 int64
date           datetime64[ns]
amount                float64
dtype: object

,purchase_id,item_id,user_id,date,amount
0,4850,39,47,2008-04-06 11:54:47,161.38
1,2356,10,45,2007-07-28 00:00:19,35.34
2,2922,117,33,2008-03-14 20:43:54,146.50
3,4929,128,77,2007-12-18 13:03:24,119.21
4,510,113,6,2007-11-09 05:18:30,105.24


## Заливка датасетов в базу данных

In [59]:
users.to_sql('users', connecton_ps, index=False, if_exists='replace', method=psql_insert_copy)
items.to_sql('items', connecton_ps, index=False, if_exists='replace', method=psql_insert_copy)
purchases.to_sql('purchases', connecton_ps, index=False, if_exists='replace', method=psql_insert_copy)

## Тестовые запросы к БД

In [60]:
sql = '''
SELECT * 
FROM users 
LIMIT 5
'''
select(sql)

,user_id,age
0,570,33.0
1,580,43.0
2,568,52.0
3,829,35.0
4,413,28.0


In [61]:
sql = '''
SELECT * 
FROM items 
LIMIT 5
'''
select(sql)

,item_id,price
0,1,161.38
1,2,35.34
2,3,146.50
3,4,119.21
4,5,105.24


In [62]:
sql = '''
SELECT * 
FROM purchases 
LIMIT 5
'''
select(sql)

,purchase_id,item_id,user_id,date,amount
0,4850,39,47,2008-04-06 11:54:47,161.38
1,2356,10,45,2007-07-28 00:00:19,35.34
2,2922,117,33,2008-03-14 20:43:54,146.50
3,4929,128,77,2007-12-18 13:03:24,119.21
4,510,113,6,2007-11-09 05:18:30,105.24


## Запросы к БД PostresSQL из заданий

### Задание А

#### Какую сумму в среднем в месяц тратят пользователи в возрастном диапазоне от 18 до 25 лет включительно

In [63]:
# ID пользователей 18 до 25 лет включительно

sql = '''
select DISTINCT user_id
from users
where age <= 25 and age > 18
ORDER BY user_id
'''
select(sql)

,user_id
0,3
1,6
2,22
3,24
4,29
...,...
161,968
162,970
163,972
164,980


In [64]:
# Выделение месяца из даты для каждого пользователя. Пример для user_id = 1

sql = '''
select *, date_trunc('month', date) as month
from purchases
where user_id = 1
order by month
'''
select(sql)

,purchase_id,item_id,user_id,date,amount,month
0,2155,85,1,2007-05-20 11:20:10,178.10,2007-05-01
1,3412,73,1,2007-06-26 00:14:54,-655.44,2007-06-01
2,2162,13,1,2007-07-05 09:02:00,10043.05,2007-07-01
3,256,69,1,2007-08-14 09:05:33,1057.14,2007-08-01
4,204,108,1,2007-08-17 13:11:51,180.39,2007-08-01
5,1043,87,1,2007-10-03 11:59:15,102.82,2007-10-01
6,4686,47,1,2007-10-07 15:37:47,1058.94,2007-10-01
7,1321,122,1,2007-10-16 18:45:51,171.60,2007-10-01
8,220,82,1,2007-10-04 18:12:59,167.49,2007-10-01
9,3689,102,1,2007-11-21 01:29:50,148.86,2007-11-01


In [65]:
# Сумма покупок по месяцам для каждого пользователя. Пример для user_id = 1

sql = '''
SELECT user_id, date_trunc('month', date) AS month, SUM(amount) AS sum_amount
FROM purchases
where user_id = 1
GROUP BY user_id, month
ORDER by month
'''
select(sql)

,user_id,month,sum_amount
0,1,2007-05-01,178.10
1,1,2007-06-01,-655.44
2,1,2007-07-01,10043.05
3,1,2007-08-01,1237.53
4,1,2007-10-01,1500.85
5,1,2007-11-01,2297.37
6,1,2007-12-01,2083.76
7,1,2008-01-01,3420.78
8,1,2008-02-01,11279.61
9,1,2008-03-01,-445.06


In [66]:
# Средняя сумма покупок в месяц для каждого пользователя.

sql = '''
SELECT user_id, AVG(sum_amount) AS average_amount
FROM (
  SELECT user_id, date_trunc('month', date) AS month, SUM(amount) AS sum_amount
  FROM purchases
  GROUP BY user_id, month
) AS subquery
GROUP BY user_id
ORDER by user_id
'''
select(sql)

,user_id,average_amount
0,1,4210.318462
1,2,3952.463636
2,3,2343.132500
3,4,5462.283000
4,5,3273.117778
...,...,...
95,96,3146.520833
96,97,4654.290000
97,98,4381.052727
98,99,1277.441111


In [67]:
# Средняя сумма покупок в месяц для каждого пользователя c учетом возраста > 18 и <= 25 лет

sql = '''
SELECT subquery.user_id, AVG(subquery.average_amount) AS average_amount_per_month
FROM (
  SELECT purchases.user_id, date_trunc('month', purchases.date) AS month, AVG(purchases.amount) AS average_amount
  FROM purchases
  INNER JOIN users ON purchases.user_id = users.user_id
  WHERE users.age > 18 AND users.age <= 25
  GROUP BY purchases.user_id, month
) AS subquery
GROUP BY subquery.user_id
ORDER BY user_id
'''
select(sql)

,user_id,average_amount_per_month
0,3,1462.791389
1,6,1817.444458
2,22,1440.044444
3,24,1551.153981
4,29,1406.616319
5,38,2814.136154
6,41,2053.703652
7,44,1665.046139
8,51,1739.057550
9,55,2682.807639


#### Какую сумму в среднем в месяц тратят пользователи в возрастном диапазоне от 26 до 35 лет включительно

In [68]:
# Средняя сумма покупок в месяц для каждого пользователя c учетом возраста > 18 и <= 25 лет

sql = '''
SELECT subquery.user_id, AVG(subquery.average_amount) AS average_amount_per_month
FROM (
  SELECT purchases.user_id, date_trunc('month', purchases.date) AS month, AVG(purchases.amount) AS average_amount
  FROM purchases
  INNER JOIN users ON purchases.user_id = users.user_id
  WHERE users.age > 26 AND users.age <= 35
  GROUP BY purchases.user_id, month
) AS subquery
GROUP BY subquery.user_id
ORDER BY user_id
'''
select(sql)

,user_id,average_amount_per_month
0,1,2132.985192
1,7,835.621667
2,8,1985.765227
3,11,1481.680121
4,13,2331.616500
5,15,2035.175361
6,22,1440.044444
7,23,1534.469667
8,25,2177.457818
9,26,999.028694


### Задание Б

#### В каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая

In [69]:
# Определение ID пользователей с возрастом > 35 лет

sql = '''
SELECT * 
FROM users 
WHERE age > 35
ORDER by user_id
'''
select(sql)

,user_id,age
0,2,57.0
1,3,74.0
2,10,61.0
3,10,62.0
4,12,54.0
...,...,...
407,986,54.0
408,987,39.0
409,988,39.0
410,988,57.0


In [70]:
# Оставляем в таблице purchases только те user_id у которых возраст > 35 лет 
# Добавляем месяц к каждому пользователю

sql = '''
SELECT *, DATE_TRUNC('month', date) AS month
FROM purchases
WHERE purchases.user_id IN (
  SELECT user_id
  FROM users
  WHERE age > 35
)
ORDER BY user_id
'''
select(sql)

,purchase_id,item_id,user_id,date,amount,month
0,899,16,2,2008-01-18 23:42:52,2071.35,2008-01-01
1,4169,173,2,2007-12-12 17:31:32,1074.19,2007-12-01
2,1134,46,2,2007-07-03 16:44:22,201.20,2007-07-01
3,2576,148,2,2007-11-24 04:59:52,1087.90,2007-11-01
4,1899,155,2,2008-01-26 20:42:30,158.90,2008-01-01
...,...,...,...,...,...,...
838,4429,169,100,2007-05-06 18:06:16,161.56,2007-05-01
839,861,109,100,2008-04-10 05:41:12,119.23,2008-04-01
840,4435,152,100,2008-03-11 09:57:00,1056.18,2008-03-01
841,807,15,100,2007-07-03 16:44:22,94.58,2007-07-01


In [71]:
# Суммируем выручку по месяцам и сортируем по убыванию общей суммы, оставляем топ-1

sql = '''
SELECT DATE_TRUNC('month', p.date) AS month, SUM(p.amount) AS total_amount
FROM purchases p
WHERE p.user_id IN (
  SELECT u.user_id
  FROM users u
  WHERE u.age > 35
)
GROUP BY month
ORDER BY total_amount DESC
LIMIT 1
'''
select(sql)

,month,total_amount
0,2008-05-01,155907.75


### Задание B

#### Какой товар дает наибольший вклад в выручку за последний год

In [72]:
# Определяем последний год

sql = '''
SELECT MAX(EXTRACT(YEAR FROM date)) AS max_year
FROM purchases
'''
select(sql)

,max_year
0,2008.0


In [73]:
# Определение ТОП-1 товаров по выручке за последний год

sql = '''
SELECT p.item_id, SUM(p.amount * i.price) AS total_revenue
FROM purchases p
JOIN items i ON p.item_id = i.item_id
WHERE EXTRACT(YEAR FROM p.date) = (
  SELECT MAX(EXTRACT(YEAR FROM date))
  FROM purchases
)
GROUP BY p.item_id
ORDER BY total_revenue DESC
LIMIT 1
'''
select(sql)

,item_id,total_revenue
0,22,3.282768e+08


### Задание Г

#### Топ-3 товаров по выручке и их доля в общей выручке за любой год

In [74]:
sql = '''
SELECT 
    p.item_id, 
    SUM(p.amount * i.price) AS total_revenue,
    SUM(p.amount * i.price) / total.total_revenue AS revenue_share
FROM purchases p
JOIN items i ON p.item_id = i.item_id
JOIN (
    SELECT 
        SUM(p.amount * i.price) AS total_revenue
    FROM purchases p
    JOIN items i ON p.item_id = i.item_id
    WHERE EXTRACT(YEAR FROM p.date) = 2008
) AS total ON 1=1
WHERE EXTRACT(YEAR FROM p.date) = 2008
GROUP BY p.item_id, total.total_revenue
ORDER BY total_revenue DESC
LIMIT 3
'''
select(sql)

,item_id,total_revenue,revenue_share
0,22,3.282768e+08,0.091825
1,109,3.119126e+08,0.087247
2,103,3.108878e+08,0.086961


SAPIENS SOLUTIONS

In [89]:
# Импорт датасета с пользователями
employees = pd.read_csv('datasets/employees.csv', sep=';')
employees.head(5)

,ID,NAME,DEPARTMENT,SALARY,EMPLOY_ID
0,1001,Ivanov,SAPIENS,100000,1001
1,1002,Petrov,SAPIENS,100000,1001
2,1003,Sidorov,TECHNOLOGIES,130000,1001
3,1004,Korotkov,TECHNOLOGIES,120000,1001
4,1005,Filev,SAPIENS,90000,1001


In [90]:
# Импорт датасета с пользователями
business_trip = pd.read_csv('datasets/business_trip.csv', sep=';')
business_trip.head(5)

,ID,EMPLOY_ID,DEP_DATE,ARRIV_DATE,TRAV_EXPENS
0,2001,1001,01.01.2014,10.01.2014,2000
1,2002,1001,01.01.2015,10.01.2015,3000
2,2003,1001,01.02.2015,10.03.2015,1500
3,2004,1001,01.06.2015,10.07.2015,2900
4,2005,1001,01.07.2015,10.08.2015,3600


In [91]:
employees.to_sql('employees', connecton_ps, index=False, if_exists='replace', method=psql_insert_copy)
business_trip.to_sql('business_trip', connecton_ps, index=False, if_exists='replace', method=psql_insert_copy)

In [92]:
sql = '''
SELECT * 
FROM employees 
LIMIT 5
'''
select(sql)

,ID,NAME,DEPARTMENT,SALARY,EMPLOY_ID
0,1001,Ivanov,SAPIENS,100000,1001
1,1002,Petrov,SAPIENS,100000,1001
2,1003,Sidorov,TECHNOLOGIES,130000,1001
3,1004,Korotkov,TECHNOLOGIES,120000,1001
4,1005,Filev,SAPIENS,90000,1001


In [93]:
sql = '''
SELECT * 
FROM business_trip 
LIMIT 5
'''
select(sql)

,ID,EMPLOY_ID,DEP_DATE,ARRIV_DATE,TRAV_EXPENS
0,2001,1001,01.01.2014,10.01.2014,2000
1,2002,1001,01.01.2015,10.01.2015,3000
2,2003,1001,01.02.2015,10.03.2015,1500
3,2004,1001,01.06.2015,10.07.2015,2900
4,2005,1001,01.07.2015,10.08.2015,3600


In [94]:
sql = '''
SELECT SUM(b.TRAV_EXPENS) AS SUM_TRAV_EXPENS
FROM employees AS e 
LEFT JOIN business_trip AS b ON e.EMPLOY_ID = b.EMPLOY_ID
WHERE e.SALARY > 90000 AND b.DEP_DATE >= '2015-06-01';'''
select(sql)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column e.employ_id does not exist
LINE 4: LEFT JOIN business_trip AS b ON e.EMPLOY_ID = b.EMPLOY_ID
                                        ^

[SQL: 
SELECT SUM(b.TRAV_EXPENS) AS SUM_TRAV_EXPENS
FROM employees AS e 
LEFT JOIN business_trip AS b ON e.EMPLOY_ID = b.EMPLOY_ID
WHERE e.SALARY > 90000 AND b.DEP_DATE >= '2015-06-01';]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [75]:
sql = '''CREATE TABLE subjects (
  id INT,
  name VARCHAR(100),
  teacher VARCHAR(100),
  cabinet INT
);
INSERT INTO subjects (Id, Name, Teacher, Cabinet)
VALUES
    (1001, 'Алгебра', 'Чичкина Ольга Александровна', 431),
    (1002, 'Биология', 'Андреева Анна Тимофеевна', 522),
    (1003, 'Химия', 'Локтева Татьяна Александровна', 123),
    (1004, 'Иностранный язык', 'Антоненко Елена Владимировна', 437),
    (1005, 'География', 'Иванова Людмила Александровна', 501),
    (1006, 'Физическая культура', 'Кривошея Тамара Яковлевна', 236),
    (1007, 'Физика', 'Котляренко Людмила Александровна', 141);  

CREATE TABLE diary (
  id INT,
  student VARCHAR(100),
  mark INT,
  subj_id INT,
  date_mark DATE
);
INSERT INTO diary (Id, Student, Mark, Subj_id, Date_mark)
VALUES
    (2001, 'Ансимова Елизавета Андреевна', 5, 1001, '2015-01-10'),
    (2002, 'Беляев Матвей Артёмович', 4, 1000, '2015-01-10'),
    (2003, 'Боленский Василий Борисович', 2, 1003,'2015-01-10'),
    (2004, 'Фазуллина Дина Фанилевна', 4, 1004, '2015-01-10'),
    (2005, 'Халитова Рената Булатовна', 5, 1009, '2015-01-10'),
    (2006, 'Маслюков Дмитрий Владимирович', 3, 1007, '2015-01-10'),
    (2007, 'Халитова Рената Булатовна', 4, 1005, '2015-01-10'),
    (2008, 'Фазуллина Дина Фанилевна', 3, 1003, '2015-01-10'),
    (2009, 'Халитова Рената Булатовна', 5, 1005, '2015-01-10'),
    (2010, 'Беляев Матвей Артёмович', 2, 1004, '2015-01-10'),
    (2011, 'Маслюков Дмитрий Владимирович', 3, 1004, '2015-01-10'),
    (2012, 'Фазуллина Дина Фанилевна', 5, 1007, '2015-01-10'),
    (2013, 'Боленский Василий Борисович', 4, 1002, '2015-01-10'),
    (2014, 'Халитова Рената Булатовна', 5, 1002, '2015-01-10'),
    (2015, 'Фазуллина Дина Фанилевна', 5, 1001, '2015-01-10'),
    (2016, 'Маслюков Дмитрий Владимирович', 4, 1001, '2015-01-10'),
    (2017, 'Ансимова Елизавета Андреевна', 4, 1005, '2015-01-10'),
    (2018, 'Боленский Василий Борисович', 3, 1003, '2015-01-10'),
    (2019, 'Ансимова Елизавета Андреевна', 5, 1007, '2015-01-10'),
    (2020, 'Беляев Матвей Артёмович', 2, 1004, '2015-01-10');

SELECT char_length(dia.student)
FROM subjects AS sub RIGHT JOIN diary AS dia
ON sub.id = dia.subj_id
GROUP BY dia.student
HAVING avg(dia.mark) LIKE (SELECT max(av.avg_mark)
                                                FROM (SELECT dia1.student, avg(dia1.mark) AS avg_mark
                                                            FROM subjects AS sub1 RIGHT JOIN diary AS dia1
                                                            ON sub1.id = dia1.subj_id
                                                            WHERE sub1.cabinet NOT LIKE '%4%' 
                                                             GROUP BY dia1.student) AS av);'''
select(sql)

TypeError: dict is not a sequence

In [ ]:
Вопрос 6: Какое значение вернет sql скрипт?

CREATE TABLE employees (
  id INT,
  name VARCHAR(50),
  department VARCHAR(50),
  salary INT
);

INSERT INTO Employees (Id, Name, Salary, department)
VALUES
    (1, 'Ivanov', 100000, 'SAPIENS'),
    (2, 'Petrov', 100000, 'SAPIENS'),
    (3, 'Sidorov', 130000, 'TECHNOLOGIES'),
    (4, 'Korotkov', 120000, 'TECHNOLOGIES'),
    (5, 'Filev', 90000, 'SAPIENS'),
    (6, 'Smirnov', 125000, 'REVOLT'),
    (7, 'Godov', 125000, 'REVOLT') ;

SELECT (AVG(ROW_NUMBER) + SUM(RANK) + SUM(DENSE_RANK))
FROM   (SELECT
      DEPARTMENT
    , SALARY
    , ROW_NUMBER() OVER(PARTITION BY DEPARTMENT ORDER BY SALARY DESC) AS 'ROW_NUMBER'
    , RANK() OVER(PARTITION BY DEPARTMENT ORDER BY SALARY DESC) AS 'RANK'
    , DENSE_RANK() OVER(PARTITION BY DEPARTMENT ORDER BY SALARY DESC) AS 'DENSE_RANK'
    FROM EMPLOYEES) AS T1
WHERE DEPARTMENT LIKE '%A__E%';